In [59]:
#Importing importing Files

import re
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_lg")

In [60]:
#Reading File Data

fileData=[]
with open("Output.txt", "r") as file:
    for line in file:
        x = re.sub("=", " = ", line)
        x = re.sub("  "," ", x)   
        fileData.append(x)

In [69]:
#Defining Patterns
#{"IN":["","s"]}
matcher = Matcher(nlp.vocab)

pattern1 = [{"LOWER":{"IN":["integer","integers"]}}]
pattern2 = [{"LOWER":{"IN":["int","ints"]}}]
pattern3 = [{"LOWER":{"IN":["float","floats"]}}]
pattern4 = [{"LOWER":{"IN":["number","numbers"]}}]
pattern5 = [{"LOWER":{"IN":["num","nums"]}}]
pattern6 = [{"LOWER":{"IN":["variable","variables"]}}]
pattern7 = [{"LOWER":{"IN":["var","vars"]}}]
pattern8 = [{"LOWER":{"IN":["double","doubles"]}}]
pattern9 = [{"LOWER":{"IN":["character","characters"]}}]
pattern10 = [{"LOWER":{"IN":["char","chars"]}}]
pattern11 = [{"LOWER":{"IN":["chr","chrs"]}}]
pattern12 = [{"LOWER":{"IN":["string","strings"]}}]
pattern13 = [{"LOWER":{"IN":["str","strs"]}}]
pattern14 = [{"LOWER":{"IN":["void","voids"]}}]
pattern15 = [{"LOWER":{"IN":["array","arrays"]}}]
pattern16 = [{"LOWER":{"IN":["list","lists"]}}]
pattern17 = [{"LOWER":{"IN":["vector","vectors"]}}]
pattern18 = [{"LOWER":{"IN":["class","classes"]}}]
matcher.add("DATATYPE", None, pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, 
            pattern8, pattern9, pattern10, pattern11, pattern12, pattern13, pattern14, pattern15, 
            pattern16, pattern17,pattern18)

pattern2_1 = [{"LOWER":"global"}]
pattern2_2 = [{"LOWER":"local"}]
matcher.add("SCOPE", None, pattern2_1, pattern2_2)

pattern3_1 = [{"LOWER":{"IN":["addition","additions"]}}]
pattern3_2 = [{"LOWER":{"IN":["add","adds"]}}]
pattern3_3 = [{"TEXT":"+"}]
pattern3_4 = [{"LOWER":{"IN":["subtraction","subtractions"]}}]
pattern3_5 = [{"LOWER":{"IN":["sub","subs"]}}]
pattern3_6 = [{"TEXT":"-"}]
pattern3_7 = [{"LOWER":{"IN":["minus","minuses"]}}]
pattern3_8 = [{"LOWER":{"IN":["multiply","multiplies"]}}]
pattern3_9 = [{"LOWER":{"IN":["mul","muls"]}}]
pattern3_10 = [{"TEXT":"*"}]
pattern3_11 = [{"LOWER":{"IN":["divide","divides"]}}]
pattern3_12 = [{"LOWER":{"IN":["div","divs"]}}]
pattern3_13 = [{"TEXT":"/"}]
pattern3_14 = [{"LOWER":{"IN":["modulo","modulus","moduluses"]}}]
pattern3_15 = [{"LOWER":{"IN":["mod","mods"]}}]
pattern3_16 = [{"TEXT":"%"}]
matcher.add("OPERATOR", None, pattern3_1, pattern3_2, pattern3_3, pattern3_4, pattern3_5, pattern3_6, pattern3_7, 
            pattern3_8, pattern3_9, pattern3_10, pattern3_11, pattern3_12, pattern3_13, pattern3_14, pattern3_15, 
            pattern3_16)

pattern4_1 = [{"LOWER":{"IN":["create","creates"]}}]
pattern4_2 = [{"LOWER":{"IN":["define","defines"]}}]
pattern4_3 = [{"LOWER":{"IN":["declare","declares"]}}]
pattern4_4 = [{"LOWER":{"IN":["set","sets"]}}]
pattern4_5 = [{"LOWER":{"IN":["print","prints"]}}]
pattern4_6 = [{"LOWER":{"IN":["display","displays"]}}]
pattern4_7 = [{"LOWER":{"IN":["show","shows"]}}]
pattern4_8 = [{"LOWER":{"IN":["get","gets"]}}]
pattern4_9 = [{"LOWER":{"IN":["input","inputs"]}}]
pattern4_10 = [{"LOWER":{"IN":["save","saves"]}}]
matcher.add("COMMAND", None, pattern4_1, pattern4_2, pattern4_3, pattern4_4, pattern4_5, pattern4_6, pattern4_7, 
            pattern4_8, pattern4_9, pattern4_10)

pattern5_1 = [{"LOWER":{"IN":["function","functions"]}}]
pattern5_2 = [{"LOWER":{"IN":["func","funcs"]}}]
pattern5_3 = [{"LOWER":{"IN":["program","programs"]}}]
pattern5_4 = [{"LOWER":{"IN":["programme","programmes"]}}]
pattern5_5 = [{"LOWER":{"IN":["prog","progs"]}}]
matcher.add("FUNC_PROG", None, pattern5_1, pattern5_2, pattern5_3, pattern5_4, pattern5_5)

In [70]:
#Creating Training Data

TRAINING_DATA = []

# Create a Doc object for each text in TEXTS
for doc in nlp.pipe(fileData):
    
    #x = re.sub("=", " = ", doc)
    #doc = re.sub("  "," ", x)
    
    # Match on the doc and create a list of matched spans
    spans = [doc[start:end] for match_id, start, end in matcher(doc)]
    
    #Modified Start
    entity_label=[]
    entity_label = [(nlp.vocab.strings[match_id]) for match_id, start, end in matcher(doc)]
    #if entity_label!=[]:
    #    print (entity_label)
    entities = [(span.start_char, span.end_char, entity_label[index]) for index,span in enumerate(spans)]
    
    #for eachWord in entities:
    #    print("Entities: {}".format(eachWord))
    
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    #Modified End
    
    '''
    entities = [(span.start_char, span.end_char, "INITIALIZATION") for span in spans]
    
    for eachWord in entities:
        print("Entities: {}".format(eachWord))
        
    # Format the matches as a (doc.text, entities) tuple
    training_example = (doc.text, {"entities": entities})
    # Append the example to the training data
    TRAINING_DATA.append(training_example)
    '''
    

#print(*TRAINING_DATA, sep="\n")


In [71]:
#Store to Data file for inspection
with open("training_data.txt","w") as o_file:
    for item in TRAINING_DATA:
        o_file.write("{}\n".format(item))

In [ ]:
#Training Loop
import random

nlp = spacy.blank("en")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

ner.add_label("DATATYPE")
ner.add_label("SCOPE")
ner.add_label("OPERATOR")
ner.add_label("COMMAND")
ner.add_label("FUNC_PROG")

# Start the training
nlp.begin_training()

# Loop for 10 iterations
for itn in range(3):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA)
    losses = {}

    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA, size=2000):
        texts = [text for text, entities in batch]
        annotations = [entities for text, entities in batch]

        # Update the model
        nlp.update(texts, annotations, losses=losses)
    print(losses)

In [66]:
from spacy import displacy

doc = nlp("Print variable i to screen")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

Print 0 5 COMMAND
variable 6 14 DATATYPE


In [67]:
doc = nlp("set numbers a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

set 0 3 COMMAND
numbers 4 11 DATATYPE


In [68]:
doc = nlp("create a function to add numbers a and b")
#doc = nlp("add global a and b")
for ent in doc.ents:
    print (ent.text, ent.start_char, ent.end_char, ent.label_)

displacy.render(nlp(doc.text), style="ent", jupyter=True)

create 0 6 COMMAND
function 9 17 OPERATOR
add 21 24 OPERATOR
numbers 25 32 DATATYPE
